In [ ]:
# Setup input parameters
from datetime import datetime as dt
dbutils.widgets.text('bg_loadtimestamp', '')
bg_loadtimestamp = dbutils.widgets.get('bg_loadtimestamp')
bg_loadtimestamp_str = bg_loadtimestamp
if not bg_loadtimestamp:
    bg_loadtimestamp = 'CAST(NULL AS Timestamp)'
else:
    bg_loadtimestamp = f"CAST('{bg_loadtimestamp}' AS Timestamp)"




In [ ]:
# Setup logging
logger = spark._jvm.org.apache.log4j.Logger.getLogger('com.bigenius-x.application')
def info(targetName, message):
    logger.info(f'{targetName}: {message}')
    print(f"{dt.now().strftime('%Y/%m/%d, %H:%M:%S')} - {targetName}: {message}")




In [ ]:
# MartDimensionLoader: Ent_stores_Mart Dimension Loader_1

try:

    operation_metrics_collection = {}
    result_df = spark.sql(f"""
    MERGE
    INTO `{marc_databricks_initiative#mart#database_name}`.`{marc_databricks_initiative#mart#schema_name}`.`dm_md_ent_stores` AS `bg_target`
    USING (
        SELECT
             `bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
            ,{bg_loadtimestamp} AS `bg_loadtimestamp`
            ,`bg_source`.`bg_rowhash` AS `bg_rowhash`
            ,`bg_source`.`store_id` AS `store_id`
        FROM `{marc_databricks_initiative#mart#database_name}`.`{marc_databricks_initiative#mart#schema_name}`.`dm_md_ent_stores_delta` AS `bg_source`
    ) AS `bg_source`
       ON (`bg_source`.`store_id` = `bg_target`.`store_id`)
    WHEN MATCHED
    THEN
        UPDATE 
        SET
             `bg_sourcesystem` = `bg_source`.`bg_sourcesystem`
            ,`bg_loadtimestamp` = `bg_source`.`bg_loadtimestamp`
            ,`bg_rowhash` = `bg_source`.`bg_rowhash`
    WHEN NOT MATCHED
    THEN
        INSERT
        (
             `bg_sourcesystem`
            ,`bg_loadtimestamp`
            ,`bg_rowhash`
            ,`store_id`
        )
        VALUES (
             `bg_source`.`bg_sourcesystem`
            ,`bg_source`.`bg_loadtimestamp`
            ,`bg_source`.`bg_rowhash`
            ,`bg_source`.`store_id`
        )
    """)
    RowCountInserted = result_df.select("num_inserted_rows").collect()[0][0]
    operation_metrics = result_df.toJSON().collect()
    operation_metrics_collection['upserttarget_{marc_databricks_initiative#mart#database_name}_{marc_databricks_initiative#mart#schema_name}_dm_md_ent_stores'] = operation_metrics

except Exception as e:
    info('DM_MD_Ent_stores_Loader', e)
    raise



In [ ]:
dbutils.notebook.exit(operation_metrics_collection)
